In [1]:
import pandas as pd 
import numpy as np 
import json
import time
import tqdm

import requests
import cloudscraper

from curl_params import HEADERS, COOKIES, get_params

SLEEP_TIME = 20

/var/folders/rn/n96315xj07s91rcy_t4k4h2r0000gn/T/ipykernel_78032/3220299706.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Parse men shoes

In [17]:
cloudscraper = cloudscraper.create_scraper()

with open('raw_data_urls.json', "r") as file:
    data = json.load(file)

page_list = [str(page_number) for page_number in range(1, 42)]
page_list = list(set(page_list) - set(list(data['men']['pages'].keys())))

for page_number in tqdm.tqdm(page_list):
    response = cloudscraper.get('https://www.ssense.com/en-id/men/shoes.json',
                         params=get_params(page_number=int(page_number)),
                         cookies=COOKIES,
                         headers=HEADERS)
    if response.status_code == 200:
        with open('raw_data_urls.json', "r") as file:
            data = json.load(file)
            if page_number not in data['men']['pages'].keys():
                data['men']['pages'][page_number] = response.json()['products']
                with open('raw_data_urls.json', "w") as file:
                    json.dump(data, file)
    else:
        print(f"response failed with status code {response.status_code}, page_number {page_number}")
    time.sleep(SLEEP_TIME)

0it [00:00, ?it/s]


# Parse women shoes

In [18]:
cloudscraper = cloudscraper.create_scraper()

with open('raw_data_urls.json', "r") as file:
    data = json.load(file)

page_list = [str(page_number) for page_number in range(1, 48)]
page_list = list(set(page_list) - set(list(data['women']['pages'].keys())))

for page_number in tqdm.tqdm(page_list):
    response = cloudscraper.get('https://www.ssense.com/en-id/women/shoes.json',
                         params=get_params(page_number=int(page_number)),
                         cookies=COOKIES,
                         headers=HEADERS)
    if response.status_code == 200:
        with open('raw_data_urls.json', "r") as file:
            data = json.load(file)
            if page_number not in data['women']['pages'].keys():
                data['women']['pages'][page_number] = response.json()['products']
                with open('raw_data_urls.json', "w") as file:
                    json.dump(data, file)
    else:
        print(f"response failed with status code {response.status_code}, page_number {page_number}")
    time.sleep(SLEEP_TIME)

100%|██████████| 47/47 [18:18<00:00, 23.38s/it]


In [19]:
def extract_brand(product: dict) -> str: 
    """extract brand from json

    Args:
        product (dict): 

    Returns:
        str: 
    """
    return product['brand']['name']['en']
    

def extract_model(product: dict) -> str: 
    """extract brand from json

    Args:
        product (dict): 

    Returns:
        str: 
    """
    return product['name']['en']
    

def extract_photos(product: dict) -> str: 
    """extract photos from json

    Args:
        product (dict): 

    Returns:
        str: 
    """
    IMAGE_PARAMS = "b_white,c_lpad,g_center,h_1412,w_940/c_scale,h_960/f_auto,dpr_2.0"
    ID = product['image'][0]
    ID = ID[67:-6]
    return [f"https://res.cloudinary.com/ssenseweb/image/upload/{IMAGE_PARAMS}/{ID}_{x}.jpg"
            for x in range(1, 5)]
    

In [20]:
extract_photos(response.json()['products'][0])

['https://res.cloudinary.com/ssenseweb/image/upload/b_white,c_lpad,g_center,h_1412,w_940/c_scale,h_960/f_auto,dpr_2.0/241807F118005_1.jpg',
 'https://res.cloudinary.com/ssenseweb/image/upload/b_white,c_lpad,g_center,h_1412,w_940/c_scale,h_960/f_auto,dpr_2.0/241807F118005_2.jpg',
 'https://res.cloudinary.com/ssenseweb/image/upload/b_white,c_lpad,g_center,h_1412,w_940/c_scale,h_960/f_auto,dpr_2.0/241807F118005_3.jpg',
 'https://res.cloudinary.com/ssenseweb/image/upload/b_white,c_lpad,g_center,h_1412,w_940/c_scale,h_960/f_auto,dpr_2.0/241807F118005_4.jpg']

# Dataset from raw data

In [21]:
with open('raw_data_urls.json', "r") as file:
    data = json.load(file)

In [22]:
df = pd.DataFrame()

for gender in data.keys():
    for page_number in tqdm.tqdm(data[gender]['pages'].keys()):
        for product in data[gender]['pages'][page_number]:
            brand = extract_brand(product=product)
            model = extract_model(product=product)
            photos = extract_photos(product=product)
            tmp_df = pd.DataFrame({"brand": [brand]*4,
                                   "model": [model]*4,
                                   "url": photos})
            df = pd.concat([df, tmp_df])

100%|██████████| 47/47 [00:10<00:00,  4.55it/s]


In [23]:
df

,brand,model,url
0,GmbH,Black Adem Boots,https://res.cloudinary.com/ssenseweb/image/upl...
1,GmbH,Black Adem Boots,https://res.cloudinary.com/ssenseweb/image/upl...
2,GmbH,Black Adem Boots,https://res.cloudinary.com/ssenseweb/image/upl...
3,GmbH,Black Adem Boots,https://res.cloudinary.com/ssenseweb/image/upl...
0,C2H4,Black Boson Boots,https://res.cloudinary.com/ssenseweb/image/upl...
...,...,...,...
3,Lauren Manoogian,Brown Welt Mules,https://res.cloudinary.com/ssenseweb/image/upl...
0,Paris Texas,Silver Lidia 70 Mules,https://res.cloudinary.com/ssenseweb/image/upl...
1,Paris Texas,Silver Lidia 70 Mules,https://res.cloudinary.com/ssenseweb/image/upl...
2,Paris Texas,Silver Lidia 70 Mules,https://res.cloudinary.com/ssenseweb/image/upl...


In [25]:
len(np.unique(df['brand'] + df['model']))

8849

In [26]:
df.to_csv('url_dataset.csv', index=None)